In [ ]:
import funkyprompt
from funkyprompt import agent, describe_function
from funkyprompt.io import add_context
from funkyprompt.ops.examples import get_context
from funkyprompt.ops.entities import AbstractVectorStoreEntry, InstructAbstractVectorStoreEntry
from funkyprompt.io.stores import VectorDataStore
from funkyprompt.io.tools.ingestion import ingest_pdf

In [ ]:
store = VectorDataStore(InstructAbstractVectorStoreEntry.create_model("BookChapters-open-ai"))
store.plot()

In [ ]:
store = VectorDataStore(InstructAbstractVectorStoreEntry.create_model("BookChapters-instruct"))
store.plot()

In [ ]:
questions = [  'What can you tell me about emergence of complexity in natural systems through evolution and the also H-4 clock for sailing and navigation',
               'What can you tell me about emergence of complexity in natural systems through evolution',
               'what can you tell me about the invention of the H-4 clock for sailing and navigation'        ]

#store(questions, refine_factor=None, limit =5)

In [ ]:
agent("What can you tell me about emergence of complexity in natural systems through evolution and the invention of the H-4 clock for sailing and navigation", 
      [store.as_function_description()])

In [ ]:
store_a = VectorDataStore(InstructAbstractVectorStoreEntry.create_model("BookChapters-instruct"))
store_b = VectorDataStore(AbstractVectorStoreEntry.create_model("BookChapters-open-ai"))

In [ ]:
store_b.plot(questions=questions, plot_type='interactive', )

In [ ]:
questions = [  'What can you tell me about emergence of complexity in natural systems through evolution and the invention of the H-4 clock for sailing and navigation',
               'What can you tell me about emergence of complexity in natural systems through evolution',
               'what can you tell me about the invention of the H-4 clock for sailing and navigation'        ]



In [ ]:
store_a.plot(questions=questions, theme='fire')
store_b.plot(questions=questions, theme='fire')

In [ ]:
#         limit: int = 5,
#         probes: int = 20,
#         refine_factor: int = 10,
#         metric="l2",
store = VectorDataStore(InstructAbstractVectorStoreEntry.create_model("InstructBookChapters"))
# agent("What can you tell me about emergence of complexity in natural systems through evolution or the invention of the clock H-4 as mentioned in the book by Sobel", 
#       [store.as_function_description("""Provides details about everything - if you are not sure about context run this function. 
#       If you are asked about multiple things at once you should pass a list questions to the query parameter and not just one. 
#       Try the compund as query 1 and then add several other questies """)])

In [ ]:
import numpy as np
e = store._embeddings('tes')
type(list(e.astype(np.float32))[0])

In [ ]:
file = "/Users/sirsh/Downloads/HiddenOrder.pdf"
file = "/Users/sirsh/Downloads/Longitude.epub"

#ingest_pdf('OpenAIBookChapters', file )

In [ ]:
store = VectorDataStore(InstructAbstractVectorStoreEntry.create_model("InstructBookChapters"))
agent("What can you tell me about emergence of complexity in natural systems through evolution or the invention of the clock", 
      [store.as_function_description("Provides details about everything - if you are not sure about context run this function")])

In [ ]:
store = VectorDataStore._load_vector_store("OpenAiBookChapters")
agent("What can you tell me about emergence of complexity in natural systems through evolution or the invention of the clock", 
      [store.as_function_description("Provides details about everything - if you are not sure about context run this function")])

In [ ]:
file = "/Users/sirsh/Documents/Archive/books/Hidden Order How Adaptation Builds Complexity (Helix Books) by John Holland (z-lib.org).pdf"

ingest_pdf('Longitude', file)

In [ ]:
a("What do you know  Emergence - How do feedback loops play a role in this ?")

In [ ]:
#its working on this interface
#do we need a generic function for stores
#do we need to do some better handling e.g. table exists exceptions etc.

In [ ]:
from funkyprompt.ops.examples import get_persons_favourite_thing_of_type, get_persons_action_if_you_know_favourite_type_of_thing
from funkyprompt.ops.utils.inspector import describe_function
fns = [describe_function(get_persons_favourite_thing_of_type), describe_function(get_persons_action_if_you_know_favourite_type_of_thing)]
#agent("What would you expect a person who likes cats to do?", fns)
#agent("John has email address john@gmail.com. What is John favourite color and his most likely action?", fns)
#agent("What is John@gmail.com's favourite color and his most likely action?", fns)


In [ ]:
#agent("What is John@gmail.com's favourite color and his most likely action? Lookup functions to get more information if you are not sure")
#agent("What is John@gmail.com's favourite color and his most likely action? ")

# Friday problem
- why does agent sometimes call the fucntion and then sometimes talk about calling the function - we can use auto or not 

In [ ]:
q= f"""
Please generate a Pydantic object and save it as Recipe.py for the data given below.
- Use only snakecase column names and alises the Pydantic fields to map from the provided data to our schema
- Simplify the schema to select up to 10 of what you consider to be the primary attributes for a Recipe.
- the Config should have a sample_url with the value {url}.
The data to use for generating the type is here however you should flatten the parent object so there are no child objects and then map the values in the root validator.
To be clear map all the data to one Pydantic object.
You should call the function to save the object or your final answer should be in the structure that the function expects.
Data:
{sample}
"""

q = f"""Please generate and save a Pydantic object for the following data using data you were trained on {sample},
        use snakecase for all new attributes and use Pydantic field alises to map from the provided sample data,
        You can use Pydantic validators to simplify attribites for the following reasons
        - Generally try to simplify all attributes to primitives or lists of primitives.
        - some attributes are lists with only one itemm You should map those to a string
        - some attributes have a dict with only one attribute. You should also map those to a string.
        - If an attribute as a list of complex types its ok to keep those and even define a child type
        - You should make all but the top 5 or 6 essential properties optional
"""

answer = agent.ask(q)
answer

In [ ]:
from funkyprompt.ops.utils.parsing import parse_fenced_code_blocks
pt = parse_fenced_code_blocks(answer, select_type='python')[0]
with open("/Users/sirsh/Downloads/rev3.py", 'w') as f:
    f.write(pt)

In [ ]:
data['video']

In [ ]:
# Plan:
- A Save pydantic "specalism" will save the file but also explain the strategy for how to make it. This is better than a generic save file

# Issues i have with LLM
- calling lookup available functions for no apparent reason
- not calling save function why

In [ ]:
!pip install diagrams

In [ ]:
from diagrams.k8s.clusterconfig import HPA
from diagrams import Diagram
with Diagram("Stateful Architecture", show=False, outformat='svg') as g: 
    h = HPA('test')
h